# Study sorting 

author: steeve.laquitaine@epfl.ch 

* Method:
    * Sorting Butterworth-filtered traces with Kilosort 3
    
* Result:
    * sorting accuracy is low and drops with increasing recording duration (10m:5%, 40m:0.07%).
    * Why?
        * **Ruled out confounds** :
            * **no true unit is silent**: validated
            * **spikes are correctly sync-ed s with concatenated traces**: visually validated by plotting both together
            * **probe rotation**: rotation w/o changing geometry? None (done)
                * observation
                    * current vertical pitch size is 2.457609e+00 due to probe rotation
                    * current horizontal pitch size is 5 due to probe rotation
                    * is typically tens of microns in kilosort's channel map example 20.0, 32.0
                    * the geometry we used is 40 and 16.
                * The pitch size should be inferred to be the vertical distance between electrodes on your probe (typically tens of um).
                    * https://github.com/MouseLand/Kilosort/issues/310                
                * running KS3 sorting on rotated coordinates to match KS3 provided examples (10m and full) -> produced very similar accuracy (a bit poorer) as actual site coordinates.
            * **minFRs filter**: Checking the effect on minFR for units and good channels -> set them to 0 (10m recording) -> produced same results as default params.
            * **lower detection threshold**: det_threshold=3 instead of 6. Marginal increase of 1% from 5% to 6% accuracy. This is the spike detection threshold in standard deviation.
            * **no correction**: produced a similar accuracy of 5%.
            * **lower last pass proj**: 4 instead of 9, produced marginally lower accuracy of -1%.
            * **lower first pass proj**: 4 instead of 9, produced marginally lower accuracy of -1%.
            * **all proj thresh**: 4 instead of 9, produced marginally lower accuracy of -1%.

    **what did not work**:
    * **no registration**: blocks=0 instead of 5.
        * we could not remove registration. KS3 crashed.                
            * "Saving results to Phy",
            *  "----------------------------------------Reference to non-existent field 'dshift'."
    * **wave_length**: has to be <81 due to GPU setup, from source code, increased to 71.
        * crashes with Unable to perform assignment because the size of the left side is 61-by-1-by-3 and the size of the right side is 71-by-3.
        * also hard coded in the source code.

#### KS3 Parameters from source code

```bash
* ops.fs = 30000;    # sample rate
* ops.fshigh = 300;  # frequency for high pass filtering (150)
* ops.Th = [9 9];  # threshold on projections (like in Kilosort1, can be different for last pass like [10 4])


% how important is the amplitude penalty (like in Kilosort1, 0 means not used, 10 is average, 50 is a lot) 
ops.lam = 20;  

% splitting a cluster at the end requires at least this much isolation for each sub-cluster (max = 1)
ops.AUCsplit = 0.8; 

% minimum spike rate (Hz), if a cluster falls below this for too long it gets removed
ops.minFR = 1/50; 

% spatial constant in um for computing residual variance of spike
ops.sigmaMask = 30; 

% threshold crossings for pre-clustering (in PCA projection space)
ops.ThPre = 8; 

% spatial scale for datashift kernel
ops.sig = 20;

% type of data shifting (0 = none, 1 = rigid, 2 = nonrigid)
ops.nblocks = 5;


%% danger, changing these settings can lead to fatal errors
% options for determining PCs
ops.spkTh           = -6;      % spike threshold in standard deviations (-6)
ops.reorder         = 1;       % whether to reorder batches for drift correction. 
ops.nskip           = 25;  % how many batches to skip for determining spike PCs

ops.GPU                 = 1; % has to be 1, no CPU version yet, sorry
% ops.Nfilt               = 1024; % max number of clusters
ops.nfilt_factor        = 4; % max number of clusters per good channel (even temporary ones)
ops.ntbuff              = 64;    % samples of symmetrical buffer for whitening and spike detection
ops.NT                  = 64*1024+ ops.ntbuff; % must be multiple of 32 + ntbuff. This is the batch size (try decreasing if out of memory). 
ops.whiteningRange      = 32; % number of channels to use for whitening each channel
ops.nSkipCov            = 25; % compute whitening matrix from every N-th batch
ops.scaleproc           = 200;   % int16 scaling of whitened data
ops.nPCs                = 3; % how many PCs to project the spikes into
ops.useRAM              = 0; % not yet available
``````

### Setup 
activate spikeinterf...

In [2]:
# import libs
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import numpy as np
from matplotlib import pyplot as plt
import spikeinterface as si
from matplotlib import pyplot as plt
import spikeinterface.full as si
import spikeinterface.extractors as se 
from spikeinterface import comparison
import spikeinterface.preprocessing as spre
from probeinterface import Probe
import MEAutility as MEA

# move to PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/preprint_2023/"
os.chdir(PROJ_PATH)

from src.nodes.utils import get_config

# SETUP DATASET COMFIG
# full recording
data_conf, param_conf = get_config("silico_neuropixels", "concatenated").values()
RAW_PATH = data_conf["probe_wiring"]["output"]
PREP_PATH = data_conf["preprocessing"]["output"]["trace_file_path"]
PREP_PATH_W = data_conf["preprocessing"]["output"]["wavelet_trace_file_path"]
SORTING_TRUE_PATH = data_conf['sorting']['simulation']['ground_truth']['output']

# 10 min recording
data_conf, _ = get_config("silico_neuropixels", "2023_10_18").values()
SORTING_TRUE_PATH_10m = data_conf['sorting']['simulation']['ground_truth']['output']
WIRED_PATH_10m = data_conf["probe_wiring"]["output"]

2024-05-01 18:46:04,429 - root - utils.py - get_config - INFO - Reading experiment config.
2024-05-01 18:46:04,656 - root - utils.py - get_config - INFO - Reading experiment config. - done
2024-05-01 18:46:04,670 - root - utils.py - get_config - INFO - Reading experiment config.
2024-05-01 18:46:04,774 - root - utils.py - get_config - INFO - Reading experiment config. - done


### Rotated probe

* Conclusion 
    * It changes the calculated horizontal and vertical pitch sizes (makes them much smaller).
    * We change the coordinates (without changing the geometry).

* check example probe rotation provided by spike interface

In [ ]:
xcoord = [
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
    43,
    11,
    59,
    27,
]
ycoord = [
    20,
    20,
    40,
    40,
    60,
    60,
    80,
    80,
    100,
    100,
    120,
    120,
    140,
    140,
    160,
    160,
    180,
    180,
    200,
    200,
    220,
    220,
    240,
    240,
    260,
    260,
    280,
    280,
    300,
    300,
    320,
    320,
    340,
    340,
    360,
    360,
    380,
    380,
    400,
    400,
    420,
    420,
    440,
    440,
    460,
    460,
    480,
    480,
    500,
    500,
    520,
    520,
    540,
    540,
    560,
    560,
    580,
    580,
    600,
    600,
    620,
    620,
    640,
    640,
    660,
    660,
    680,
    680,
    700,
    700,
    720,
    720,
    740,
    740,
    760,
    760,
    780,
    780,
    800,
    800,
    820,
    820,
    840,
    840,
    860,
    860,
    880,
    880,
    900,
    900,
    920,
    920,
    940,
    940,
    960,
    960,
    980,
    980,
    1000,
    1000,
    1020,
    1020,
    1040,
    1040,
    1060,
    1060,
    1080,
    1080,
    1100,
    1100,
    1120,
    1120,
    1140,
    1140,
    1160,
    1160,
    1180,
    1180,
    1200,
    1200,
    1220,
    1220,
    1240,
    1240,
    1260,
    1260,
    1280,
    1280,
    1300,
    1300,
    1320,
    1320,
    1340,
    1340,
    1360,
    1360,
    1380,
    1380,
    1400,
    1400,
    1420,
    1420,
    1440,
    1440,
    1460,
    1460,
    1480,
    1480,
    1500,
    1500,
    1520,
    1520,
    1540,
    1540,
    1560,
    1560,
    1580,
    1580,
    1600,
    1600,
    1620,
    1620,
    1640,
    1640,
    1660,
    1660,
    1680,
    1680,
    1700,
    1700,
    1720,
    1720,
    1740,
    1740,
    1760,
    1760,
    1780,
    1780,
    1800,
    1800,
    1820,
    1820,
    1840,
    1840,
    1860,
    1860,
    1880,
    1880,
    1900,
    1900,
    1920,
    1920,
    1940,
    1940,
    1960,
    1960,
    1980,
    1980,
    2000,
    2000,
    2020,
    2020,
    2040,
    2040,
    2060,
    2060,
    2080,
    2080,
    2100,
    2100,
    2120,
    2120,
    2140,
    2140,
    2160,
    2160,
    2180,
    2180,
    2200,
    2200,
    2220,
    2220,
    2240,
    2240,
    2260,
    2260,
    2280,
    2280,
    2300,
    2300,
    2320,
    2320,
    2340,
    2340,
    2360,
    2360,
    2380,
    2380,
    2400,
    2400,
    2420,
    2420,
    2440,
    2440,
    2460,
    2460,
    2480,
    2480,
    2500,
    2500,
    2520,
    2520,
    2540,
    2540,
    2560,
    2560,
    2580,
    2580,
    2600,
    2600,
    2620,
    2620,
    2640,
    2640,
    2660,
    2660,
    2680,
    2680,
    2700,
    2700,
    2720,
    2720,
    2740,
    2740,
    2760,
    2760,
    2780,
    2780,
    2800,
    2800,
    2820,
    2820,
    2840,
    2840,
    2860,
    2860,
    2880,
    2880,
    2900,
    2900,
    2920,
    2920,
    2940,
    2940,
    2960,
    2960,
    2980,
    2980,
    3000,
    3000,
    3020,
    3020,
    3040,
    3040,
    3060,
    3060,
    3080,
    3080,
    3100,
    3100,
    3120,
    3120,
    3140,
    3140,
    3160,
    3160,
    3180,
    3180,
    3200,
    3200,
    3220,
    3220,
    3240,
    3240,
    3260,
    3260,
    3280,
    3280,
    3300,
    3300,
    3320,
    3320,
    3340,
    3340,
    3360,
    3360,
    3380,
    3380,
    3400,
    3400,
    3420,
    3420,
    3440,
    3440,
    3460,
    3460,
    3480,
    3480,
    3500,
    3500,
    3520,
    3520,
    3540,
    3540,
    3560,
    3560,
    3580,
    3580,
    3600,
    3600,
    3620,
    3620,
    3640,
    3640,
    3660,
    3660,
    3680,
    3680,
    3700,
    3700,
    3720,
    3720,
    3740,
    3740,
    3760,
    3760,
    3780,
    3780,
    3800,
    3800,
    3820,
    3820,
    3840,
    3840,
]
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(5, 10))
plt.plot(xcoord, ycoord, ".")

# label sites
for ix in range(len(xcoord)):
    plt.text(
        xcoord[ix],
        ycoord[ix],
        "%s" % (str(chan_map[0][ix])),
        size=5,
        zorder=100,
        color="r",
    )

* check our probe's horizontal and vertical pitch as calculated in Kilosort

In [ ]:
# get Recording extractors
Prepro = si.load_extractor(PREP_PATH)

# channel ids
chan_map = np.arange(0, 384, 1)

# coordinates
xcoord_2 = Prepro.get_channel_locations()[:, 0]
ycoord_2 = Prepro.get_channel_locations()[:, 1]

# init plot
fig, ax = plt.subplots(1, 1, figsize=(5, 10))
ax.plot(xcoord_2, ycoord_2, ".", markersize=2)

# label sites
for ix in range(len(xcoord_2)):
    ax.text(
        xcoord_2[ix],
        ycoord_2[ix],
        "%s" % (str(chan_map[ix])),
        size=5,
        zorder=100,
        color="r",
    )

# vertical pitch
dmin = np.median(np.diff(np.unique(ycoord_2)))
print(dmin)

# horizontal pitch
yunq = np.unique(ycoord_2)
mxc = np.zeros(len(yunq))
for j in range(len(yunq)):
    xc = xcoord_2[ycoord_2 == yunq[j]]
    if len(xc) > 1:
        mxc[j] = np.median(np.diff(np.sort(xc)))
dminx = np.max([5, np.median(mxc)])
print(dminx)

* rotate probe and save

In [ ]:
# channel ids
chan_map = np.arange(0, 384, 1)

# create new coordinates *****
ProbeNeuropix384 = MEA.return_mea("Neuropixels-384")
coords = ProbeNeuropix384.positions[:, [1, 2]]
new_coords = np.vstack(
    [
        coords[:96, :][::-1],
        coords[96 : 2 * 96, :][::-1],
        coords[2 * 96 : 3 * 96, :][::-1],
        coords[3 * 96 : 4 * 96, :][::-1],
    ]
)

# create 2D probe with the locations *****
ProbeVivo = Probe(ndim=2, si_units="um")
ProbeVivo.set_contacts(
    positions=new_coords,
    shapes="square",
    shape_params={
        "width": 6.0,
    },
)
ProbeVivo.create_auto_shape(probe_type="tip")

# set contact and channel ids (mapped to the rows of the locations array)
ProbeVivo.set_contact_ids(np.arange(0, 384, 1))  # default order in Kilosort
ProbeVivo.set_device_channel_indices(Prepro.get_channel_ids())

# update RecordingExtractor *****
Prepro = Prepro.set_probe(ProbeVivo)

# plot to check
new_coords = Prepro.get_channel_locations()

fig, ax = plt.subplots(1, 1, figsize=(5, 10))
ax.plot(new_coords[:, 0], new_coords[:, 1], ".", markersize=2)

# label sites
for ix in range(len(coords)):
    ax.text(
        new_coords[ix, 0],
        new_coords[ix, 1],
        "%s" % (str(chan_map[ix])),
        size=5,
        zorder=100,
        color="r",
    )

# do we replicate vertical and horizontal pitches seen in
# kilosort's example channel map files? YES.
ycoords = new_coords[:, 1]
xcoords = new_coords[:, 0]

# vertical pitch
dmin = np.median(np.diff(np.unique(ycoords)))
print(dmin)

# horizontal pitch
yunq = np.unique(ycoords)
mxc = np.zeros(len(yunq))
for j in range(len(yunq)):
    xc = xcoords[ycoords == yunq[j]]
    if len(xc) > 1:
        mxc[j] = np.median(np.diff(np.sort(xc)))
dminx = np.max([5, np.median(mxc)])
print(dminx)

## Save wired recording takes 50 mins
# Prepro.save(
#     folder="/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/0_silico/neuropixels/concatenated_campaigns/preprocessed/traces_rotd_coord",
#     format="binary",
#     overwrite=True,
#     n_jobs=2,
#     chunk_memory="40G",
#     progress_bar=True,
# )

In [ ]:
# full recording
Sortingrotd_coord_si01005 = si.load_extractor(
    "/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/0_silico/neuropixels/concatenated_campaigns/SortingKS3_rotd_coord_si01005/"
)
print(
    "single unit count",
    sum(Sortingrotd_coord_si01005.get_property("KSLabel") == "good"),
)
print("mua count:", sum(Sortingrotd_coord_si01005.get_property("KSLabel") == "mua"))


# 10 min recording
Sortingrotd_coord_si01005_10m = si.load_extractor(
    "/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/sorting/0_silico/neuropixels_lfp_10m_384ch_hex0_40Khz_2023_10_18/be011315-9555-493e-a59c-27f42d1058ed/SortingKS3_rotd_coord_si01005_10m"
)
print(
    "single unit count",
    sum(Sortingrotd_coord_si01005_10m.get_property("KSLabel") == "good"),
)
print("mua count:", sum(Sortingrotd_coord_si01005_10m.get_property("KSLabel") == "mua"))

In [ ]:
# butterworth 10m ...
SortingTrue10m = si.load_extractor(SORTING_TRUE_PATH_10m)

# accuracies
matching = comparison.compare_sorter_to_ground_truth(
    SortingTrue10m, Sortingrotd_coord_si01005_10m, exhaustive_gt=True
)
accuracy_brotd_coord_si01005_10m = matching.agreement_scores.max(axis=1).sort_values(
    ascending=False
)
# butterworth
SortingB10m = si.load_extractor(
    "/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/sorting/0_silico/neuropixels_lfp_10m_384ch_hex0_40Khz_2023_10_18/be011315-9555-493e-a59c-27f42d1058ed/SortingKS3/"
)
matching = comparison.compare_sorter_to_ground_truth(
    SortingTrue10m, SortingB10m, exhaustive_gt=True
)
accuracy_b10m = matching.agreement_scores.max(axis=1).sort_values(ascending=False)


# butterworth rotd full
SortingTrue = si.load_extractor(SORTING_TRUE_PATH)
matching = comparison.compare_sorter_to_ground_truth(
    SortingTrue, Sortingrotd_coord_si01005, exhaustive_gt=True
)
accuracy_brotd_coord_si01005 = matching.agreement_scores.max(axis=1).sort_values(
    ascending=False
)


print("\nBUTTERWORTH 10m")
print(
    "sorting accuracy: ",
    sum(accuracy_b10m.values >= 0.8) / len(accuracy_b10m.values),
)
print("# of well detected: ", sum(accuracy_b10m.values >= 0.8))
print("number of true units: ", len(SortingTrue10m.unit_ids))


print("\nBUTTERWORTH 10m rotd 10m....")
print(
    "sorting accuracy: ",
    sum(accuracy_brotd_coord_si01005_10m.values >= 0.8)
    / len(accuracy_brotd_coord_si01005_10m.values),
)
print("# of well detected: ", sum(accuracy_brotd_coord_si01005_10m.values >= 0.8))
print("number of true units: ", len(SortingTrue10m.unit_ids))

# ******
print("\nBUTTERWORTH rotd full")
print(
    "sorting accuracy: ",
    sum(accuracy_brotd_coord_si01005.values >= 0.8)
    / len(accuracy_brotd_coord_si01005.values),
)
print("# of well detected: ", sum(accuracy_brotd_coord_si01005.values >= 0.8))
print("number of true units: ", len(SortingTrue.unit_ids))

### no minFR

In [ ]:
Sorting_noMinFR = si.load_extractor(
    "/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/sorting/0_silico/neuropixels_lfp_10m_384ch_hex0_40Khz_2023_10_18/be011315-9555-493e-a59c-27f42d1058ed/SortingKS3_si01005_noMinFr/"
)
SortingTrue10m = si.load_extractor(SORTING_TRUE_PATH_10m)
matching = comparison.compare_sorter_to_ground_truth(
    SortingTrue10m, Sorting_noMinFR, exhaustive_gt=True
)
acc = matching.agreement_scores.max(axis=1).sort_values(ascending=False)

print(
    "sorting accuracy: ",
    sum(acc.values >= 0.8) / len(acc.values),
)
print("# of well detected: ", sum(acc.values >= 0.8))
print("number of true units: ", len(SortingTrue10m.unit_ids))

### Wave length

sbatch cluster/sorting/marques_silico/test/sort_ks3_si01005_wv_len.sbatch

### Lower detection threshold

sbatch cluster/sorting/marques_silico/test/sort_ks3_si01005_detthres.sbatch

In [7]:
path = "/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/sorting/0_silico/neuropixels_lfp_10m_384ch_hex0_40Khz_2023_10_18/be011315-9555-493e-a59c-27f42d1058ed/SortingKS3_si01005_detthres/"

# det_thresh=3
Sorting = si.load_extractor(path)
# true
SortingTrue = si.load_extractor(SORTING_TRUE_PATH_10m)
matching = comparison.compare_sorter_to_ground_truth(
    SortingTrue, Sorting, exhaustive_gt=True
)
# accuracy
acc = matching.agreement_scores.max(axis=1).sort_values(ascending=False)
acc_sort = matching.agreement_scores.max(axis=0).sort_values(ascending=False)

# print
print(
    "sorting accuracy: ",
    sum(acc.values >= 0.8) / len(acc.values),
)
print(
    "sorting accuracy (sort view): ",
    sum(acc_sort.values >= 0.8) / len(acc_sort.values),
)
print("# of well detected: ", sum(acc.values >= 0.8))
print("number of true units: ", len(SortingTrue.unit_ids))

/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/envs/spikinterf0_100_5/lib/python3.9/site-packages/spikeinterface/core/base.py:1079: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.96.1 is recommended
  warnings.warn(


sorting accuracy:  0.06183206106870229
sorting accuracy (sort view):  0.09712230215827339
# of well detected:  81
number of true units:  1310


### No correction

In [6]:
path = "/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/sorting/0_silico/neuropixels_lfp_10m_384ch_hex0_40Khz_2023_10_18/be011315-9555-493e-a59c-27f42d1058ed/SortingKS3_si01005_nocorr/"

# det_thresh=3
Sorting = si.load_extractor(path)

# true
SortingTrue = si.load_extractor(SORTING_TRUE_PATH_10m)
matching = comparison.compare_sorter_to_ground_truth(
    SortingTrue, Sorting, exhaustive_gt=True
)
# accuracy
acc = matching.agreement_scores.max(axis=1).sort_values(ascending=False)
acc_sort = matching.agreement_scores.max(axis=0).sort_values(ascending=False)

# print
print(
    "sorting accuracy: ",
    sum(acc.values >= 0.8) / len(acc.values),
)
print(
    "sorting accuracy (sort view): ",
    sum(acc_sort.values >= 0.8) / len(acc_sort.values),
)
print("# of well detected: ", sum(acc.values >= 0.8))
print("number of true units: ", len(SortingTrue.unit_ids))

/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/envs/spikinterf0_100_5/lib/python3.9/site-packages/spikeinterface/core/base.py:1079: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.96.1 is recommended
  warnings.warn(


sorting accuracy:  0.05648854961832061
sorting accuracy:  0.09035409035409035
# of well detected:  74
number of true units:  1310


### Lower last pass proj. thresh.

sbatch cluster/sorting/marques_silico/test/sort_ks3_si01005_lstpssproj4.sbatch

In [8]:
path = "/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/sorting/0_silico/neuropixels_lfp_10m_384ch_hex0_40Khz_2023_10_18/be011315-9555-493e-a59c-27f42d1058ed/SortingKS3_si01005_lstpssproj4/"

# det_thresh=3
Sorting = si.load_extractor(path)

# true
SortingTrue = si.load_extractor(SORTING_TRUE_PATH_10m)
matching = comparison.compare_sorter_to_ground_truth(
    SortingTrue, Sorting, exhaustive_gt=True
)
# accuracy
acc = matching.agreement_scores.max(axis=1).sort_values(ascending=False)
acc_sort = matching.agreement_scores.max(axis=0).sort_values(ascending=False)

# print
print(
    "sorting accuracy: ",
    sum(acc.values >= 0.8) / len(acc.values),
)
print(
    "sorting accuracy (sort view): ",
    sum(acc_sort.values >= 0.8) / len(acc_sort.values),
)
print("# of well detected: ", sum(acc.values >= 0.8))
print("number of true units: ", len(SortingTrue.unit_ids))

/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/envs/spikinterf0_100_5/lib/python3.9/site-packages/spikeinterface/core/base.py:1079: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.96.1 is recommended
  warnings.warn(


sorting accuracy:  0.06030534351145038
sorting accuracy (sort view):  0.06899563318777292
# of well detected:  79
number of true units:  1310


### Lower first pass proj. thresh.

In [9]:
path = "/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/sorting/0_silico/neuropixels_lfp_10m_384ch_hex0_40Khz_2023_10_18/be011315-9555-493e-a59c-27f42d1058ed/SortingKS3_si01005_fstpssproj4/"

# det_thresh=3
Sorting = si.load_extractor(path)

# true
SortingTrue = si.load_extractor(SORTING_TRUE_PATH_10m)
matching = comparison.compare_sorter_to_ground_truth(
    SortingTrue, Sorting, exhaustive_gt=True
)
# accuracy
acc = matching.agreement_scores.max(axis=1).sort_values(ascending=False)
acc_sort = matching.agreement_scores.max(axis=0).sort_values(ascending=False)

# print
print(
    "sorting accuracy: ",
    sum(acc.values >= 0.8) / len(acc.values),
)
print(
    "sorting accuracy (sort view): ",
    sum(acc_sort.values >= 0.8) / len(acc_sort.values),
)
print("# of well detected: ", sum(acc.values >= 0.8))
print("number of true units: ", len(SortingTrue.unit_ids))

/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/envs/spikinterf0_100_5/lib/python3.9/site-packages/spikeinterface/core/base.py:1079: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.96.1 is recommended
  warnings.warn(


sorting accuracy:  0.04351145038167939
sorting accuracy (sort view):  0.0581039755351682
# of well detected:  57
number of true units:  1310


### All proj thresh to 4

In [10]:
path = "/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/sorting/0_silico/neuropixels_lfp_10m_384ch_hex0_40Khz_2023_10_18/be011315-9555-493e-a59c-27f42d1058ed/SortingKS3_si01005_allpssproj4/"

# det_thresh=3
Sorting = si.load_extractor(path)

# true
SortingTrue = si.load_extractor(SORTING_TRUE_PATH_10m)
matching = comparison.compare_sorter_to_ground_truth(
    SortingTrue, Sorting, exhaustive_gt=True
)
# accuracy
acc = matching.agreement_scores.max(axis=1).sort_values(ascending=False)
acc_sort = matching.agreement_scores.max(axis=0).sort_values(ascending=False)

# print
print(
    "sorting accuracy: ",
    sum(acc.values >= 0.8) / len(acc.values),
)
print(
    "sorting accuracy (sort view): ",
    sum(acc_sort.values >= 0.8) / len(acc_sort.values),
)
print("# of well detected: ", sum(acc.values >= 0.8))
print("number of true units: ", len(SortingTrue.unit_ids))

/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/envs/spikinterf0_100_5/lib/python3.9/site-packages/spikeinterface/core/base.py:1079: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.96.1 is recommended
  warnings.warn(


sorting accuracy:  0.04122137404580153
sorting accuracy (sort view):  0.025210084033613446
# of well detected:  54
number of true units:  1310
